In [16]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from sklearn.preprocessing import StandardScaler

In [3]:
work_dir = os.getcwd()
df = pd.read_csv(f"{work_dir}/coffee_classification.csv")

In [4]:
df.head()

,Coffee Type,1,2,3,4,5,6,7,8,9,...,592,593,594,595,596,597,598,599,600,601
0,Tauro,0.948504,0.947795,0.946445,0.944988,0.943754,0.943384,0.944066,0.945799,0.948352,...,0.282806,0.281890,0.281222,0.280815,0.280693,0.280811,0.281193,0.281853,0.282838,0.284274
1,Tauro,0.905532,0.904839,0.903945,0.902695,0.901430,0.900893,0.901221,0.902677,0.904882,...,0.258335,0.257492,0.256907,0.256648,0.256645,0.256880,0.257425,0.258203,0.259317,0.260800
2,Tauro,0.784601,0.784103,0.783307,0.782516,0.781724,0.781432,0.781957,0.783278,0.785293,...,0.222330,0.221602,0.221146,0.220880,0.220885,0.221101,0.221590,0.222324,0.223326,0.224650
3,Tauro,0.720050,0.719402,0.718323,0.717160,0.716094,0.715715,0.716034,0.717204,0.718991,...,0.198238,0.197453,0.196903,0.196654,0.196652,0.196831,0.197252,0.197917,0.198817,0.200071
4,Tauro,0.820835,0.820500,0.819734,0.818661,0.817800,0.817529,0.818071,0.819510,0.821659,...,0.232847,0.232013,0.231440,0.231195,0.231231,0.231466,0.231987,0.232698,0.233674,0.234985


EDA

In [8]:
print(df.columns)

Index(['Coffee Type', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '592', '593', '594', '595', '596', '597', '598', '599', '600', '601'],
      dtype='object', length=602)


In [7]:
print(df.describe())

               1          2          3          4          5          6  \
count  70.000000  70.000000  70.000000  70.000000  70.000000  70.000000   
mean    0.730464   0.729880   0.728958   0.727939   0.727150   0.726963   
std     0.097779   0.097676   0.097532   0.097392   0.097286   0.097257   
min     0.522417   0.522132   0.521493   0.520754   0.520109   0.519794   
25%     0.659202   0.658728   0.657851   0.656829   0.655862   0.655480   
50%     0.728594   0.727914   0.726955   0.725886   0.725179   0.724936   
75%     0.783336   0.782698   0.781683   0.780743   0.780144   0.780152   
max     0.966410   0.965667   0.964509   0.963394   0.962561   0.962427   

               7          8          9         10  ...        592        593  \
count  70.000000  70.000000  70.000000  70.000000  ...  70.000000  70.000000   
mean    0.727574   0.728983   0.730981   0.733238  ...   0.211783   0.211042   
std     0.097326   0.097515   0.097792   0.098090  ...   0.031218   0.031116   
min 

It looks like the data is required to scale as it contains high to low value variations from mean. and the very first column is for classification.
Now, maybe there might be outliers in the data and have to check it in the next steps.
For outlier detection, Savitzky-Golay smoothing technique with some degree of derivatvie if required will be used including the adjustment of the window-size

In [14]:
X = df.values[:, 1:]
# scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [19]:
# smoothing the data
window = 5
X_smooth_0 = savgol_filter(X, window_length=window, polyorder=0)
X_smooth_1 = savgol_filter(X, window_length=window, polyorder=1)
X_smooth_2 = savgol_filter(X, window_length=window, polyorder=2)

wavelength = np.linspace(1100, 2300, X.shape[1])

In [27]:
plt.figure(figsize=(12, 9))
frequency_interval = np.arange(0, X., 1)
plt.plot(wavelength[frequency_interval], X[frequency_interval], "b", label="No Smoothing")
plt.show()

IndexError: index 601 is out of bounds for axis 0 with size 601

<Figure size 1200x900 with 0 Axes>